In [ ]:
import plotly.io as pio

import Config.constants as cnst
import Analysis.figures as figs
from Analysis.DetectorComparisonAnalyzer import DetectorComparisonAnalyzer

pio.renderers.default = "notebook"

In [ ]:
DATASET = "Lund2013"
COL_MAPPER = lambda col: col[:col.index("ector")] if "ector" in col else col
VERBOSE = True
SHOW_DISTRIBUTIONS = True

In [ ]:
%%capture --no-stdout

_, events, _, event_matches, comparison_columns = DetectorComparisonAnalyzer.preprocess_dataset(DATASET,
                                                                         column_mapper=COL_MAPPER,
                                                                         verbose=VERBOSE)

# events = events.map(lambda cell: [e for e in cell if e.event_label == cnst.EVENT_LABELS.FIXATION])
# event_matches = event_matches.map(
#     lambda cell: {k: v for k, v in cell.items() if k.event_label == cnst.EVENT_LABELS.FIXATION}
#     if cell is not None else None)

comparison_columns

In [ ]:
fixation_metrics = DetectorComparisonAnalyzer.analyze(events, event_matches,
                                    ignore_events={v for v in cnst.EVENT_LABELS if v != cnst.EVENT_LABELS.FIXATION},
                                    verbose=VERBOSE)
event_features = fixation_metrics["Event Features"]
event_matching_ratios = fixation_metrics["Event Matching Ratios"]
event_matching_feature_diffs = fixation_metrics["Event Matching Feature Diffs"]

# Fixations' Feature Distributions

In [ ]:
print(f"Available fixation features: {list(event_features.keys())}")

In [ ]:
if SHOW_DISTRIBUTIONS:
    for feature in event_features.keys():
        if feature not in {"Duration"}:
            continue
        title = f"{DATASET.upper()}:\t\tFixations' {feature.title()} Distribution"
        figs.distributions_grid(
            data=event_features[feature],
            title=title,
            show_counts=feature == "Counts",
        ).show()

# Matched-Fixations' Feature Differences

In [ ]:
print(f"Available matched-fixation feature differences: {list(event_matching_feature_diffs.keys())}")

In [ ]:
if SHOW_DISTRIBUTIONS:
    figs.distributions_grid(
        data=event_matching_ratios["Match Ratio"][comparison_columns],
        title=f"{DATASET.upper()}:\t\tFixation-Matching Ratios",
        pdf_min_val=0,
        pdf_max_val=100,
        column_title_mapper=lambda col: f"{col[0]}→{col[1]}"
    ).show()

In [ ]:
if SHOW_DISTRIBUTIONS:
    for feature in event_matching_feature_diffs.keys():
        if feature not in ['Duration', 'Onset Jitter', 'Overlap Time', 'L2 Timing Jitter', 'Offset Jitter', 'IoU']:
            continue
        figs.distributions_grid(
            data=event_matching_feature_diffs[feature][comparison_columns],
            title=f"{DATASET.upper()}:\t\tMatched-Fixations' {feature.title()} Distribution",
            column_title_mapper=lambda col: f"{col[0]}→{col[1]}",
            pdf_min_val=0 if feature == "IoU" else None,
            pdf_max_val=1 if feature == "IoU" else None,
        ).show()